# Scraping de URLs Legítimas de Bancos Españoles

**Autor:** Alexis Zapico  
**Fecha:** 20/07/2025

## Objetivo
Obtener un dataset de URLs legítimas de las principales entidades bancarias de España.  
El resultado servirá como referencia para la validación y entrenamiento del modelo de detección de phishing.  
Este notebook **solo recoge y guarda los datos crudos**; la limpieza y análisis se realizarán en etapas posteriores.


## 3. Bancos a scrapear

A continuación se muestran los bancos objetivo del scraping.

-  BBVA
-  Santander
-  CaixaBank
-  Sabadell
-  Bankinter
-  Unicaja Banco
-  Ibercaja
-  Abanca
-  Kutxabank
-  Cajamar
-  Caja Rural
-  Openbank
-  EVO Banco
-  ING España

**Criterio de selección:**  
Principales entidades bancarias en España, priorizando aquellas con más usuarios y exposición a ciberataques.


### Importamos librerías

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, os
import time
from urllib.parse import urljoin, urlparse
from time import sleep


### Diccionario con los bancos a scrapear y sus urls principales

In [115]:
bancos = {
    "BBVA": "https://www.bbva.es",
    "Santander": "https://www.bancosantander.es",
    "CaixaBank": "https://www.caixabank.es",
    "Sabadell": "https://www.bancsabadell.com",
    "Bankinter": "https://www.bankinter.com",
    "Unicaja Banco": "https://www.unicajabanco.es",
    "Ibercaja": "https://www.ibercaja.es",
    "Abanca": "https://www.abanca.com",
    "Kutxabank": "https://www.kutxabank.es",
    "Cajamar": "https://www.cajamar.es",
    "Caja Rural": "https://www.ruralvia.com",
    "Openbank": "https://www.openbank.es",
    "EVO Banco": "https://www.evobanco.com",
    "ING España": "https://www.ing.es"
}


## Función para scraping

In [10]:
from urllib.parse import urljoin, urlparse  # Importa funciones para unir y analizar URLs

def obtener_urls(base_url):
    """
    Descarga la página principal del banco y extrae las URLs internas.
    Acepta URLs del dominio principal y de sus subdominios (pero no de dominios externos).
    """
    try:
        # Realiza la petición HTTP a la URL base del banco con un tiempo máximo de espera de 10 segundos
        response = requests.get(base_url, timeout=10)

        # Parsea el HTML recibido para poder analizarlo y extraer enlaces
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtiene el dominio principal del banco (por ejemplo, bbva.es)
        dominio_banco = urlparse(base_url).netloc

        # Elimina 'www.' al inicio si lo hubiera, para igualar subdominios y dominio principal
        if dominio_banco.startswith("www."):
            dominio_base = dominio_banco[4:]
        else:
            dominio_base = dominio_banco

        # Set para almacenar URLs únicas
        urls = set()

        # Itera por todos los enlaces <a href="...">
        for link in soup.find_all('a', href=True):
            href = link['href']  # Extrae el href del enlace
            href = urljoin(base_url, href)  # Convierte relativo a absoluto

            # Obtiene el dominio del enlace encontrado
            dominio_href = urlparse(href).netloc
            if dominio_href.startswith("www."):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añade la URL solo si es del dominio principal o subdominio (y no externo)
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Devuelve todas las URLs únicas como lista
        return list(urls)

    except Exception as e:
        # Si ocurre algún error, informa por pantalla y devuelve lista vacía
        print(f"Error al acceder a {base_url}: {e}")
        return []


## Guardamos los resultados

In [137]:
resultados = []
for nombre, url_base in bancos.items():
    print(f'Scraping banco: {nombre}')
    urls_banco = obtener_urls(url_base) # Devuelve una lsita de urls
    for url in urls_banco:
        resultados.append({'banco': nombre, 'url': url})
    time.sleep(2)

Scraping banco: BBVA
Scraping banco: Santander
Scraping banco: CaixaBank
Scraping banco: Sabadell
Scraping banco: Bankinter
Scraping banco: Unicaja Banco
Scraping banco: Ibercaja
Scraping banco: Abanca
Scraping banco: Kutxabank
Scraping banco: Cajamar
Scraping banco: Caja Rural
Scraping banco: Openbank
Scraping banco: EVO Banco
Scraping banco: ING España


In [144]:
# Creamos la carpeta "raw", si ya existe, no hace nda
os.makedirs('../data/raw', exist_ok=True)


# Convertimos la lista de resultados en un DataFrame
df_scrap = pd.DataFrame(resultados)
# Guardamos el DataFrame en un archivo CSV
df_scrap.to_csv('../data/raw/urls_legitimas_crudo.csv', index=False)
# Mensaje de confirmación y muestra rápida
print(f"Datos guardados correctamente en data/raw/urls_legitimas_crudo.csv")
print(df_scrap.head())

Datos guardados correctamente en data/raw/urls_legitimas_crudo.csv
           banco                                                url
0      CaixaBank  https://www.caixabank.es/particular/home/parti...
1      CaixaBank  https://www.caixabank.es/particular/general/co...
2      CaixaBank  https://www.caixabank.es/particular/general/co...
3  Unicaja Banco                    https://uniblog.unicajabanco.es
4  Unicaja Banco  https://www.unicajabanco.es/es/particulares/cu...


## 22/07/2025 - Scraping de bancos pendientes

**Objetivo del día:**  
Realizar scraping de las páginas oficiales de los siguientes bancos, que aún no tienen URLs registradas en la base de datos del proyecto.

**Bancos a procesar:**  
- Banco de España  
- Bankia (ahora CaixaBank)  
- BBK / Laboral Kutxa  
- ING Direct  
- Santander Consumer Finance  
- Banca March  
- Targobank

**Observaciones iniciales:**  
- Registrar el número de URLs obtenidas para cada banco.
- Anotar cualquier incidencia técnica (captchas, bloqueos, páginas sin enlaces, etc.).
- Actualizar la tabla maestra y el diario de campo al finalizar.

---


“Se han eliminado de la lista de scraping de URLs legítimas las entidades fusionadas o absorbidas (Bankia, BBK/Laboral Kutxa, ING Direct) para evitar redundancia y ruido. Solo se considerarán para la fase de phishing si detecto que siguen siendo suplantadas.”

In [7]:
bancos_nuevos ={
    "Banco de España": "https://www.bde.es",
    "Santander Consumer Finance": "https://www.santanderconsumer.es",
    "Banca March": "https://www.bancamarch.es",
    "Targobank": "https://www.targobank.es"
}

    

In [13]:
## Aplicamos la función para obtener las urls

In [12]:
resultados_nuevos = []
for nombre, url in bancos_nuevos.items():
    print(f'Scrapeando{nombre}...')
    try:
        urls_banco = obtener_urls(url)
        print(f'{nombre}: {len(urls_banco)} URLs obtenidas.')
        for url_encontrada in urls_banco:
            resultados_nuevos.append({'banco': nombre, 'url': url_encontrada})
    except Exception as e:
        print(f'Error en {nombre}: {e}')

ScrapeandoBanco de España...
Banco de España: 671 URLs obtenidas.
ScrapeandoSantander Consumer Finance...
Santander Consumer Finance: 0 URLs obtenidas.
ScrapeandoBanca March...
Banca March: 85 URLs obtenidas.
ScrapeandoTargobank...
Targobank: 0 URLs obtenidas.


In [14]:
## Guardamos los resultados en un dataframe

In [17]:
df_scrap_2 = pd.DataFrame(resultados_nuevos)


In [15]:
## Guardamos el dataframe en un csv 

In [23]:
df_scrap_2.to_csv('../data/raw/urls_legitimas_22_07_2025.csv', index=False)